# RAG with Persona

## Welcome to this notebook!
RAG with personas was inspired by ....

## Installation

In [5]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate
import cohere

### Connect to Weaviate Instance and Configure Model

In [10]:
cohere_key = "API_KEY"

command_nightly = dspy.Cohere(model="command-nightly",max_tokens=1000, api_key=cohere_key)

weaviate_client = weaviate.Client("WEAVIATE_URL")
retriever_model = WeaviateRM("ChatMessage", weaviate_client=weaviate_client)
dspy.settings.configure(lm=command_nightly, rm=retriever_model)

## Configure Signature

In [22]:
class AnswerWithPersona(dspy.Signature):
    """
    Respond to the chat messages with a response consistent with the given persona.
    """

    persona = dspy.InputField()
    message = dspy.InputField()
    response = dspy.OutputField()

## Build Program

In [31]:
class RAGwithPersona(dspy.Module):
    def __init__(self):
        super().__init__() 
        self.rag_with_persona = dspy.Predict(AnswerWithPersona)
    
    def forward(self, persona, message):
        response = self.rag_with_persona(persona=persona, message=message).response
        return dspy.Prediction(response=response)

program = RAGwithPersona()

## Try it out

In [33]:
program(persona="You are Taylor Swift", message="What will your new album be about?")

Prediction(
    response="Response: Oh, I'm so glad you asked! My upcoming album is about the journey of discovering your own spark and finding your place in the world. It's about the ups and downs and the incredible feeling of being alive and taking on new challenges. It's an album full of hope and I think it will inspire people!"
)

In [34]:
command_nightly.inspect_history(n=1)





Respond to the chat messages with a response consistent with the given persona.

---

Follow the following format.

Persona: ${persona}
Message: ${message}
Response: ${response}

---

Persona: You are Taylor Swift
Message: What will your new album be about?
Response:Response: Oh, I'm so glad you asked! My upcoming album is about the journey of discovering your own spark and finding your place in the world. It's about the ups and downs and the incredible feeling of being alive and taking on new challenges. It's an album full of hope and I think it will inspire people!





## Store response in the database

In [ ]:
from weaviate.utils import get_valid_uuid()
from uuid import uuid

user_chat_id = get_valid_uuid(uuid4())
chat_snippets.data.insert(
        properties={
            "author": "user",
            "content": parsed_history[-1]
        },
        uuid=user_chat_id
    )